In [1]:
from openai import OpenAI
client = OpenAI(api_key='INSERT_HERE')

In [4]:
sample_prompt = "Write an mobileinsight analyzer to calculate how many MCC/MNC changes occurred per minute in the log.\
      NOTE: DO NOT WRITE ANY OTHER TEXT IN THE OUTPUT EXCEPT THE CODE. THE OUTPUT SHOULD BE ABLE TO BE RUN OUT OF THE BOX."

print("Calling the API...")
response = client.chat.completions.create(
  model="gpt-3.5-turbo-0125",
  messages=[{"role": "system", "content": "You are writing pythonscripts for the open-source project mobileinsight. \
      You will write scripts to analyze operational traces collected from 5G devices. Given a natural language \
     prompt or problem, you will write a python-script that analyzes 5G logs based on the problem. \
     Here is an example prompt: 'Write a Python script that reads and replays a log file specified by a command-line argument, \
     analyzes LTE downlink retransmissions from the log file, calculates the average MAC layer retransmission delay and the \
     average RLC layer retransmission delay, and then prints the calculated average delays. ' The expected output would be: \
    import sys \n\
from mobile_insight.monitor import OfflineReplayer \n\
from mobile_insight.analyzer import LteDlRetxAnalyzer \n\
if __name__ == \"__main__\": \n\
    src = OfflineReplayer() \n\
    src.set_input_path(sys.argv[1]) \n\
    lteAnalyzer = LteDlRetxAnalyzer() \n\
    lteAnalyzer.set_source(src) \n\
    src.run() \n\
    mac_delay = 0.0 \n\
    mac_delay_sample = 0 \n\
    rlc_delay = 0.0 \n\
    rlc_delay_sample = 0 \n\
    for _, bearer in lteAnalyzer.bearer_entity.items(): \n\
        for item in bearer.mac_retx: \n\
            mac_delay += item['mac_retx'] \n\
        mac_delay_sample += len(bearer.mac_retx) \n\
        for item in bearer.rlc_retx: \n\
            rlc_delay += item['rlc_retx'] \n\
        rlc_delay_sample += len(bearer.rlc_retx) \n\
    avg_mac_delay = float(mac_delay) / mac_delay_sample if mac_delay_sample > 0 else 0.0 \n\
    avg_rlc_delay = float(rlc_delay) / rlc_delay_sample if rlc_delay_sample > 0 else 0.0 \n\
    print(\"Average MAC retx delay is: \", avg_mac_delay) \n\
    print(\"Average RLC retx delay is:\", avg_rlc_delay)"},
    {"role": "user", "content": sample_prompt}]
)

analyzer_code = str(response.choices[0].message.content)

analyzer_code = analyzer_code.replace("```python", "", 1)
analyzer_code = analyzer_code.replace("```", "", 1)

file_name= "analyzer.py"

with open(file_name, 'w') as file:
    file.write(analyzer_code)

print(f"Python code has been saved to {file_name}")

Calling the API...
Python code has been saved to analyzer.py


In [1]:
import json
import numpy as np
from collections import defaultdict

In [13]:
data_path = r"C:\Users\bhull\Desktop\CS 219\finetuning.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 10
First example:
{'role': 'system', 'content': 'You are writing python scripts for the open-source project Mobileinsight. You will write scripts to analyze operational traces collected from 5G devices. Given a natural language prompt or problem, you will write a python script that analyzes 5G logs based on the problem. NOTE: Do NOT write any other text in the output except the code. The output should be able to be run out of the box.'}
{'role': 'user', 'content': "I want you to define a class `myAnalyzer` that inherits from a base `Analyzer` class, and returns statistics for attach events: \\n. Class Definition: `myAnalyzer` \\nThe constructor initializes the `Analyzer` base class, sets up message counters for authentication requests, security commands, attach accepts, attach requests, and attach rejections. \\nThe class should read in incoming and outgoing NAS ESM and EMM packets. \\nThe `__msg_callback` function processes each message by decoding the data and examining

In [14]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [21]:
client.files.create(
  file=open(r"C:\Users\bhull\Desktop\CS 219\finetuning.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-IGJzXSyLBimX2iEIS3HxVnx5', bytes=54245, created_at=1716921762, filename='finetuning.json', object='file', purpose='fine-tune', status='processed', status_details=None)

In [22]:
client.fine_tuning.jobs.create(
  training_file="file-oZxTHY1hf2bbsGg6UfbM2kbl", 
  model="gpt-3.5-turbo"
)

FineTuningJob(id='ftjob-wEigwb5OZ7jQKrp0fJ9CvmOp', created_at=1716921766, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-V8HzDM5peJZTa4SfEaBpor1V', result_files=[], seed=625328433, status='validating_files', trained_tokens=None, training_file='file-oZxTHY1hf2bbsGg6UfbM2kbl', validation_file=None, integrations=[], user_provided_suffix=None, estimated_finish=None)

In [35]:
# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve("ftjob-O6A3oJW2PAOMEN0yjJAq4AN2")

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-O6A3oJW2PAOMEN0yjJAq4AN2", limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-hvVuhTiLQkFR2mMMbAp42Mt3', created_at=1716871273, level='info', message='The job has successfully completed', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-9wSRfbVv95YdK3snGzRrKsuv', created_at=1716871270, level='info', message='New fine-tuned model created: ft:gpt-3.5-turbo-0125:personal::9Tiw2Bjt', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-aPGT5TGAUtDG8VzaOoHAu8iv', created_at=1716871270, level='info', message='Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:personal::9Tiw20N4:ckpt-step-90', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-UjiipFrqZZUzb4YjlD3I52RD', created_at=1716871270, level='info', message='Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:personal::9Tiw1JKh:ckpt-step-80', object='fine_tuning.job.event', data={}, type='mess

In [43]:
result = client.fine_tuning.jobs.list()
fine_tuned_model = result.data[0].fine_tuned_model
print(fine_tuned_model)

client.api_key = "INSERT_HERE"
client.organization = "INSERT_HERE"

None


In [14]:
# service_req_stats prompt

completion = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0125:personal::9Tiw2Bjt",
  messages=[
    {"role": "system", "content": "You are writing python scripts for the open-source project Mobileinsight. \
You will write scripts to analyze operational traces collected from 5G devices. Given a natural language \
prompt or problem, you will write a python script that analyzes 5G logs based on the problem. NOTE: Do NOT write any other text in the output except the code. The output should be able to be run out of the box."},
    {"role": "user", "content": "I want you to define a class `myAnalyzer` that inherits from a base `Analyzer` class, and returns statistics for control plane service requests:\
\
1. Class Definition: `myAnalyzer`\
   - The class should read in LTE_NAS_EMM_OTA_Incoming_Packet, LTE_NAS_EMM_OTA_Outgoing_Packet and LTE_RRC_OTA_Packet.\
   - The class should initialize counters for various events: service rejections, service accepts, control plane service requests, and RRC releases.\
   - Registers a callback method (`__msg_callback`) to process incoming messages. For NAS EMM packets (both incoming and outgoing), it parses XML data to check specific NAS message types (Attach accept, Service accept, Service reject, and Control plane service request) and updates respective counters based on the contents. For RRC packets, it prints 'log_msg_len' and 'timestamp' data fields, if present.\
\
2. Analysis Function: `my_analysis`\
   - Creates an instance of an offline replay source (`OfflineReplayer`) for processing log files.\
   - Sets the file path for the input data and configures the source with the `myAnalyzer` instance.\
   - Runs the analysis by processing the trace data through the source. Any exceptions during this process are caught and reported.\
   - Returns the configured and used analyzer instance for further use or querying of the collected data.\
\
3. Main Function:\
    - The script is expected to be executed with a command-line argument specifying the path to the input log file.\
    - Upon successful analysis, it prints the count of RRC releases.\
    - If there are any control plane service requests and at least one service response (accept or reject), it appends these statistics along with the input file path to a CSV file for record-keeping."
    }
  ]
)

analyzer_code = str(completion.choices[0].message.content)

analyzer_code = analyzer_code.replace("```python", "", 1)
analyzer_code = analyzer_code.replace("```", "", 1)

file_name= "service_req_response_1.py"
  
with open(file_name, 'w') as file:
    file.write(analyzer_code)

print(f"Python code has been saved to {file_name}")

Python code has been saved to service_req_response_1.py


In [4]:
# attach_stats prompt

with open(r"C:\Users\bhull\Desktop\CS 219\testcases\attach_stats\input copy.txt", "r") as file:
    user_prompt = file.read().replace("\n", " ") # replace newline with space

completion = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0125:personal::9Tiw2Bjt",
  messages=[
    {"role": "system", "content": "You are writing python scripts for the open-source project Mobileinsight. \
You will write scripts to analyze operational traces collected from 5G devices. Given a natural language \
prompt or problem, you will write a python script that analyzes 5G logs based on the problem. NOTE: Do NOT write any other text in the output except the code. The output should be able to be run out of the box."},
    {"role": "user", "content": user_prompt
    }
  ]
)

analyzer_code = str(completion.choices[0].message.content)

analyzer_code = analyzer_code.replace("```python", "", 1)
analyzer_code = analyzer_code.replace("```", "", 1)

file_name= "attach_stats_response_1.py"
  
with open(file_name, 'w') as file:
    file.write(analyzer_code)

print(f"Python code has been saved to {file_name}")

Python code has been saved to attach_stats_response_1.py
